In [105]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [94]:
url = 'https://drive.google.com/file/d/1kM__riNHRPx5GsyuOH3yhiql3OZvwmuP/view?usp=sharing'
path_tidy = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path_tidy)

In [152]:
from datetime import datetime as dt
from datetime import timedelta 
def convert_to_float(X):
    if X == "Goal":
        return 1.0
    else:
        return 0.0
def convert_to_total_seconds(X):
    t_ = dt.strptime(X, '%M:%S')
    delta = timedelta(minutes=t_.minute,seconds=t_.second)
    return delta.total_seconds()

def preprocess(df, features,standarize=False):
    df_shuffled = df.copy()
 
  
    
    df_shuffled['result_event']=df_shuffled['result_event'].apply(convert_to_float)
    Y = df_shuffled['result_event']
    
    
    
    
    df_shuffled["empty_net"].fillna(False,inplace=True)
    df_shuffled["empty_net"]=df_shuffled["empty_net"].map({True:1,False:0})
    df_shuffled["rebound"]=df_shuffled["rebound"].map({True:1,False:0})
    df_shuffled["strength"].fillna(0.0,inplace=True)
    
    
    
    df_shuffled = df_shuffled[features]
    df_shuffled = df_shuffled.dropna()
    if 'periodTime' in features:
        df_shuffled['periodTime']=df_shuffled['periodTime'].apply(convert_to_total_seconds)
    
    if 'shot_type' in features:
        df_shuffled['shot_type'] = df_shuffled['shot_type'].dropna()
        df_shuffled = pd.get_dummies(df_shuffled,columns=['shot_type'])    
        
    if 'last_event_type' in features:
        df_shuffled = pd.get_dummies(df_shuffled,columns=['last_event_type'])
    
    X = df_shuffled
    if standarize:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    
    return X, Y.values,df_shuffled


In [153]:
list_features = ['empty_net', 'periodTime','period', 'x_coord', 'y_coord','distance','angle','shot_type',\
    'last_event_type', 'last_x_coord', 'last_y_coord','distance_from_last', 'seconds_since_last', \
        'rebound', 'angle_change','speed']

X, Y ,df_ =  preprocess(df,list_features,True)

In [154]:
df_.isna().sum().sum()

0

In [155]:
len(df_)

400554

In [149]:
print(df_.isna().head())
df_.columns[df_.isna().any()].tolist()

   empty_net  periodTime  period  x_coord  y_coord  distance  angle  \
0      False       False   False    False    False     False  False   
1      False       False   False    False    False     False  False   
2      False       False   False    False    False     False  False   
3      False       False   False    False    False     False  False   
4      False       False   False    False    False     False  False   

   last_x_coord  last_y_coord  distance_from_last  ...  \
0         False         False               False  ...   
1         False         False               False  ...   
2         False         False               False  ...   
3         False         False               False  ...   
4         False         False               False  ...   

   last_event_type_Official Challenge  last_event_type_Penalty  \
0                               False                    False   
1                               False                    False   
2                         

['x_coord',
 'y_coord',
 'distance',
 'angle',
 'last_x_coord',
 'last_y_coord',
 'distance_from_last',
 'speed']

In [132]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[-0.83343799,  0.67557622],
       [-0.63635562, -0.85793878],
       [ 2.07364637, -0.60743591],
       ...,
       [-1.20704035, -0.80085975],
       [-1.0793159 , -0.90682568],
       [-0.88923237, -0.43197777]])